In [13]:
cd /home/jordan/150827/

/home/jordan/150827


In [18]:
ls

CM763-A_debarcoded.fasta                    preprocess_reads.cfg
CM763-A_S42_L008_R1_001.fastq.processed.gz


### Preprocessing

#### Remove linker and short reads

In [5]:
run /usr/local/bin/preprocess_reads.py CM763-A_S42_L008_R1_001.fastq.gz --min-length 26 --no-gzip --linker TATGCCGTCT --max-length 40

In [14]:
cd processed_01-20-2016_jordan.0/

/home/jordan/150827/processed_01-20-2016_jordan.0


#### Collapse identical reads

 Note: output is fasta not fastq

In [17]:
!fastx_collapser  -i CM763-A_S42_L008_R1_001.fastq.processed.gz -o CM763-A_collapsed.fasta

#### Remove the first 6 nt (barcode) before alignment

In [19]:
run /home/jordan/CodeBase/Strip6.py CM763-A_collapsed.fasta

### Align to rRNA

In [5]:
!bowtie -p10 -v2 -m1 --un CM763-A_rRNA_un /home/jordan/GENOMES/crypto_rRNA -f CM763-A_collapsed_debarcoded.fasta --sam CM763-A_rRNA_al.sam

# reads processed: 5047312
# reads with at least one reported alignment: 572924 (11.35%)
# reads that failed to align: 4465876 (88.48%)
# reads with alignments suppressed due to -m: 8512 (0.17%)
Reported 572924 alignments to 1 output stream(s)


### Align to genome

In [15]:
!bowtie -p10 -v2 -M1 --best --max CM763-A_multi --un CM763-A_un /home/jordan/GENOMES/Crypto_for_gobs -f CM763-A_rRNA_un --sam CM763-A.sam

# reads processed: 4474388
# reads with at least one reported alignment: 3256882 (72.79%)
# reads that failed to align: 1112989 (24.87%)
# reads with alignments sampled due to -M: 104517 (2.34%)
Reported 3256882 alignments to 1 output stream(s)


### Make BAM files and sort

In [16]:
cd ../BAMFILES

/home/jordan/150827/BAMFILES


In [17]:
!samtools view -Sbo CM763-A.bam ../processed_01-20-2016_jordan.0/CM763-A.sam

[samopen] SAM header is present: 14 sequences.


In [18]:
!samtools sort CM763-A.bam CM763-A_sorted

In [19]:
!samtools index CM763-A_sorted.bam

### Make bedGraph for IGV

In [21]:
!genomeCoverageBed -ibam CM763-A_sorted.bam -g /home/jordan/GENOMES/crypto_for_bedgraph.genome -bg > CM763-A.bedgraph

### Run crunchBAM to count different read types

(great for SP datasets, but useful for others as well)

In [22]:
ls

CM763-A.bam  CM763-A.bedgraph  CM763-A_sorted.bam  CM763-A_sorted.bam.bai


In [24]:
run /home/jordan/CodeBase/python-hm/crunchBAM.py process --db=/home/jordan/GENOMES/H99GOBS.db --dir=./JEB002 --cores=10 --transcripts=/home/jordan/CodeBase/python-hm/CNA3.gff3 --find_pileups --count_total_reads --count_exonic_reads --count_splice_site_reads *sorted.bam

{'--cen': None,
 '--cores': '10',
 '--count_exonic_reads': True,
 '--count_junction_reads': False,
 '--count_splice_site_reads': False,
 '--count_total_reads': False,
 '--db': '/home/jordan/GENOMES/H99GOBS.db',
 '--dir': './JEB002',
 '--fasta': None,
 '--find_pileups': 0,
 '--gff': None,
 '--output': None,
 '--species': None,
 '--transcripts': '/home/jordan/CodeBase/python-hm/CNA3.gff3',
 '<bamfile>': ['CM763-A_sorted.bam'],
 'build_database': False,
 'process': True}
count_exonic_reads
saving output..
counting exonic reads
Empty DataFrame
Columns: [CM763-A_sorted.bam]
Index: []

[0 rows x 1 columns]
Coverage database saved in  ./JEB002transcript_cover_db
Database constructed and saved
done
CM763-A_sorted.bam
